In [17]:
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [18]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import json
import requests
import time

In [19]:
# Chargement du jeu de données
df = pd.read_csv('valeursfoncieres-2020.txt',sep="|",decimal=",")
df.head()
#print(df.columns)

C:\Users\olivier\anaconda\envs\Geo\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (14,18,23,24,26,28,41) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Code service CH,Reference document,1 Articles CGI,2 Articles CGI,3 Articles CGI,4 Articles CGI,5 Articles CGI,No disposition,Date mutation,Nature mutation,...,Surface Carrez du 5eme lot,Nombre de lots,Code type local,Type local,Identifiant local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,07/01/2020,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,T,NaN,1061.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,02/01/2020,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,BT,NaN,85.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,02/01/2020,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,T,NaN,1115.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,02/01/2020,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,T,NaN,1940.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,02/01/2020,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,T,NaN,1148.0


In [20]:
# focus on the address datas
ad = df[['No voie', 'Type de voie', 'Voie', 'Code postal','Commune', 'Code departement', 'Code commune']]
ad.shape

(2459560, 7)

In [21]:
ad.head()

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune
0,NaN,NaN,FORTUNAT,1250.0,CEYZERIAT,1,72
1,NaN,NaN,TERRES DES CINQ SAULES,1290.0,LAIZ,1,203
2,NaN,NaN,BOIS DU CHAMP RION,1290.0,LAIZ,1,203
3,NaN,NaN,EN COROBERT,1290.0,LAIZ,1,203
4,NaN,NaN,TERRES DES CINQ SAULES,1290.0,LAIZ,1,203


In [22]:
# On supprime les adresses dupliquées pour éviter d'interroger les API plusieurs fois avec exactement la même adresse
ad = ad.drop_duplicates()
ad.shape

(1059450, 7)

In [23]:
ad.head(10)

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune
0,NaN,NaN,FORTUNAT,1250.0,CEYZERIAT,1,72
1,NaN,NaN,TERRES DES CINQ SAULES,1290.0,LAIZ,1,203
2,NaN,NaN,BOIS DU CHAMP RION,1290.0,LAIZ,1,203
3,NaN,NaN,EN COROBERT,1290.0,LAIZ,1,203
6,NaN,RUE,DE LA CHARTREUSE,1960.0,PERONNAS,1,289
7,NaN,NaN,CHAMP PORTIER,1370.0,VAL-REVERMONT,1,426
9,NaN,NaN,CHAMPEL,1270.0,COLIGNY,1,108
11,5367.0,NaN,LORETTE,1270.0,COLIGNY,1,108
13,31.0,RUE,COMTE DE LA TEYSSONNIERE,1000.0,BOURG-EN-BRESSE,1,53
14,6.0,AV,DU CHAMP DE FOIRE,1000.0,BOURG-EN-BRESSE,1,53


In [24]:
# Traitement des codes postaux (transformés en INT sauf les NA)

ad['Code postal'] = ad['Code postal'].fillna(-1).astype(int).astype(str).replace('-1', np.nan)
ad.head()

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune
0,NaN,NaN,FORTUNAT,1250,CEYZERIAT,1,72
1,NaN,NaN,TERRES DES CINQ SAULES,1290,LAIZ,1,203
2,NaN,NaN,BOIS DU CHAMP RION,1290,LAIZ,1,203
3,NaN,NaN,EN COROBERT,1290,LAIZ,1,203
6,NaN,RUE,DE LA CHARTREUSE,1960,PERONNAS,1,289


In [25]:
# On formate les adresses
# tools functions : nan values management
def escapeNan(row):
    val = str(row)
    if val == "nan":
        val = " "
    else:
        val = val.replace('.0','') # if from float values
        val = val + " "
    return val

In [26]:
# Création d'un champ "Ville" pour interroger les coordonnées GPS lorsque le champ Adresse ne donne rien
ad['Ville'] = ad['Code postal'] + " " + ad['Commune']
ad.head()

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune,Ville
0,NaN,NaN,FORTUNAT,1250,CEYZERIAT,1,72,1250 CEYZERIAT
1,NaN,NaN,TERRES DES CINQ SAULES,1290,LAIZ,1,203,1290 LAIZ
2,NaN,NaN,BOIS DU CHAMP RION,1290,LAIZ,1,203,1290 LAIZ
3,NaN,NaN,EN COROBERT,1290,LAIZ,1,203,1290 LAIZ
6,NaN,RUE,DE LA CHARTREUSE,1960,PERONNAS,1,289,1960 PERONNAS


In [27]:
# Création du champ "addressName" qui sera utilisée dans la requête API
ad['Adresse'] = ad.apply(lambda row: 
    escapeNan(row['No voie'])
    + escapeNan(row['Type de voie'])         
    + escapeNan(row['Voie'])
    + escapeNan(row['Code postal']).zfill(5) # pour respecter le format des codes postaux
    + str(row['Commune'])
    , axis = 1).str.strip()

In [28]:
ad.head()

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune,Ville,Adresse
0,NaN,NaN,FORTUNAT,1250,CEYZERIAT,1,72,1250 CEYZERIAT,FORTUNAT 1250 CEYZERIAT
1,NaN,NaN,TERRES DES CINQ SAULES,1290,LAIZ,1,203,1290 LAIZ,TERRES DES CINQ SAULES 1290 LAIZ
2,NaN,NaN,BOIS DU CHAMP RION,1290,LAIZ,1,203,1290 LAIZ,BOIS DU CHAMP RION 1290 LAIZ
3,NaN,NaN,EN COROBERT,1290,LAIZ,1,203,1290 LAIZ,EN COROBERT 1290 LAIZ
6,NaN,RUE,DE LA CHARTREUSE,1960,PERONNAS,1,289,1960 PERONNAS,RUE DE LA CHARTREUSE 1960 PERONNAS


In [29]:
ad['Ville'] = ad['Ville'].replace('-', ' ', regex=True).astype(str)
ad.head()

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune,Ville,Adresse
0,NaN,NaN,FORTUNAT,1250,CEYZERIAT,1,72,1250 CEYZERIAT,FORTUNAT 1250 CEYZERIAT
1,NaN,NaN,TERRES DES CINQ SAULES,1290,LAIZ,1,203,1290 LAIZ,TERRES DES CINQ SAULES 1290 LAIZ
2,NaN,NaN,BOIS DU CHAMP RION,1290,LAIZ,1,203,1290 LAIZ,BOIS DU CHAMP RION 1290 LAIZ
3,NaN,NaN,EN COROBERT,1290,LAIZ,1,203,1290 LAIZ,EN COROBERT 1290 LAIZ
6,NaN,RUE,DE LA CHARTREUSE,1960,PERONNAS,1,289,1960 PERONNAS,RUE DE LA CHARTREUSE 1960 PERONNAS


In [30]:
ad.to_csv('export_dvf_addresses_2020.csv')